# IE440 HOMEWORK 2

## Group 11

importing needed basic libraries

In [54]:
import pandas as pd
import numpy as np
import random as rn

reading data and converting them into numpy arrays

In [15]:
coordinates = pd.read_csv("coordinates_11.csv", header = None, names = ["x_coor","y_coor"])
coordinates.index += 1

In [16]:
a = coordinates.to_numpy()

In [17]:
header_list = list(range(101))
header_list.remove(0)

In [18]:
costs = pd.read_csv("costs_11.csv", header = None, names = header_list )
costs.index += 1

In [19]:
c = costs.to_numpy()

In [20]:
demands = pd.read_csv("demands_11.csv",header = None, names = ["Demands"])
demands.index += 1

In [21]:
h = demands.to_numpy()

understanding how many customer and facility nodes

In [22]:
print(len(h))
print(len(c))

100
50


creating `C` matrix with multiplying `c*h` matrices

In [23]:
C = [ [ 0 for i in range(100) ] for j in range(50) ]

In [24]:
for j in range(len(h)):
    for i in range(len(c)):
        C[i][j] = h[j][0] * c[i][j]

### Question 1

creating empty lists and arrays for single facility porblem algorithm

In [13]:
empty_list = [[0.],[0.]]
x = np.array(empty_list)
sum_of_numerator = [[0.],[0.]]
sum_of_denominator = [[0.],[0.]]

first facility(index 0) has been choosed to solve single facility algorithm

In [25]:
for i in range(2):
    for j in range(100):
        sum_of_numerator[i] += C[0][j]*a[j][i]
        sum_of_denominator[i] += C[0][j]
    x[i] = sum_of_numerator[i] / sum_of_denominator[i]

In [26]:
x
# x1 coordinate of the facility is 50.57
# x2 coordinate of the facility is 47.94

array([[50.5674239 ],
       [47.94188058]])

for the first facility(index 0), algorithm found the coordinates as
- x1 = 50.57
- x2 = 47.94

Because this is algorithm solved for single facility all customers assigned to one facility.

to calculate distance, I created `squared_oclid_distance` function.

In [27]:
def squared_oclid_distance(x_t, a_j):
    return (x_t[0] - a_j[0])**2 + (x_t[1] - a_j[1])**2

calculating objective value `Z`

In [62]:
Z = 0.

for j in range(100):
    Z += C[0][j]*squared_oclid_distance(x,a[j])

float(Z)

203258461.52568173

As we can see algorithm try to satisfy all the demands from a single facility, objective value is quite big (around 203.25 million)

### Question 2:

For weiszfeld algorithm, because algorithm is a recursive algortrihm, we need to stop the algorithm after some iteration when it converge to some point. Here, we used `T` as `10`, which means algorithm compute for 10 iteration which is enough for converge to some point.

Also we need a initial position of the facility, we choosed x1 = 25 and x2 = 25

In [39]:
T = 10
weiszfeld_x = np.zeros(shape = (T-1,2))
weiszfeld_x[0][0] = 25
weiszfeld_x[0][1] = 25

Also because weiszfed using oclid distance, we created another distance algortihm called `oclid_distance` for it.

In [46]:
def oclid_distance(w_x, a_j):
    return np.sqrt( (w_x[0] - a_j[0])**2 + (w_x[1] - a_j[1])**2 )

We write a dynamic function for weiszfeld algorithm, which takes `T` and initial coordinates of the facility. 

In [57]:
def weiszfeld_dynamic(T, default_val_x, default_val_y): 
    weiszfeld_x = np.zeros(shape = (T, 2))
    weiszfeld_x[0][0] = default_val_x
    weiszfeld_x[0][1] = default_val_y
    for t in range(1, T):
        for k in range(len(weiszfeld_x[0])):
            nom = 0
            denom = 0
            for j in range(len(C[0])):
                d = oclid_distance(weiszfeld_x[t-1], a[j])
                nom += (C[0][j]) * (a[j][k]) / d
                denom += (C[0][j]) / d
            weiszfeld_x[t][k] = nom / denom                                                   
    return weiszfeld_x[T-1]

In [59]:
x = weiszfeld_dynamic(20, 25., 25.)
print(x)

[47.42530424 47.42526402]


In [64]:
x = weiszfeld_dynamic(100, 25., 25.)
print(x)

[47.42559065 47.42545599]


We run the algorithm for T = 20 and T = 100, as we can see both coordinates almost the same. 

For the first facility(index 0), weiszfeld algorithm calculate the facility coordinates as:

- x1 = 47.42
- x2 = 47.42

Also, because it solves single facility problem, all the customers assign to one facility.

Now, we can calculate the objective value `Z_weiszfeld`

In [65]:
Z_weiszfeld = 0.


for j in range(100):
    Z_weiszfeld += C[0][j]*oclid_distance(x,a[j])

float(Z_weiszfeld)

4664198.305211185

As we aspect, Weiszfeld algorithm is more effective to solve single facility problems, our objective function decreased significantly (around 4.66 million)

### Question 3

To run ALA algortihm first we need to allocate each customers to a facility randomly so we write a function for it called `initialization`.

In [92]:
rn_list = []
def initialization():
    for i in range(len(C[0])):
        n = rn. randint(0,len(C)-1)
        rn_list.append(n)
    return rn_list
initialization()
print(rn_list)


[8, 39, 30, 25, 20, 48, 43, 39, 21, 5, 32, 37, 35, 8, 7, 47, 11, 32, 6, 22, 0, 16, 40, 13, 15, 18, 36, 35, 9, 24, 14, 38, 35, 35, 49, 38, 27, 46, 14, 0, 6, 23, 29, 9, 31, 15, 44, 33, 45, 21, 45, 40, 14, 25, 38, 13, 6, 41, 17, 32, 13, 16, 24, 3, 12, 10, 48, 20, 12, 27, 33, 48, 14, 9, 11, 26, 18, 0, 11, 20, 6, 26, 5, 26, 17, 10, 40, 31, 4, 46, 12, 20, 24, 40, 27, 12, 14, 45, 49, 34]


`initialization` function returns 100 integers (customers) between 0-49 which is number of facilities.

Now, we allocate customers to facilities in the `y` matrix.

In [93]:
y = np.zeros((50,100))
for i in range(len(rn_list)):
    k = rn_list[i]
    y[k][i] = 1.

Also, we need to initialize a matrix for the coordiantes of the facilities called `M`

In [94]:
M = np.zeros(shape = (50,2))

Here is the algorithm for m single facility location problem

In [95]:
for k in range(50): 
    sum_of_numerator = [[0.],[0.]]
    sum_of_denominator = [[0.],[0.]]
    for i in range(2):
        for j in range(100):
            sum_of_numerator[i] += C[k][j]*a[j][i]*y[k][j]
            sum_of_denominator[i] += C[k][j]*y[k][j]
            if(sum_of_denominator[i] != 0):
                M[k][i] = sum_of_numerator[i] / sum_of_denominator[i]

Algorithm locates the facilities but random list does not include all facilities. Therefore, we locate the facilities which does not have customer to the coordinate (50,50)

In [96]:
# putting facilities which does not have any customers in (50,50)
for i in range(len(M)):
    if(M[i][0] == 0 and M[i][1] == 0):
        M[i][0] = 50.
        M[i][1] = 50.

In order to compare them with the final solution of ALA algorithm, we calculate first objective function. 

In [97]:
# Calculating first objective value
Z_ALA = 0.
for i in range(len(C)):
    for j in range(len(C[0])):
        if(M[i][0] != 0):
            d = squared_oclid_distance(M[i], a[j])
        else:
            d = 0
        Z_ALA += y[i][j]*C[i][j]*d

In [98]:
print(Z_ALA)

121875501.81347221


Because allocation is totally random objective value is very big. 

Next step of the algorithm, assign customers to nearest(least costly) facilities. While we are calculating least costly facility, we multiply distance with `C` matrix which is demand*(unit cost)

In [99]:
#assign customers to nearest(least costly) facilities
for j in range(len(C[0])):
    mincost = 100000000000
    for i in range(len(C)):
        cost_i_j = squared_oclid_distance(M[i], a[j])*C[i][j]
        if(cost_i_j < mincost):
            mincost = cost_i_j
            for k in range(len(C)):
                if(y[k][j] == 1):
                    y[k][j] = 0
                if(k == i):
                    y[k][j] = 1

After initializations we can put the algorithm in a loop which is going to end if there is no improvement in the objective value. 

In [100]:
eps = 0.001
Z_ALA_new = 0.
while(1):
      
    
    # solving m single facility location problem 
    for k in range(50): 
        sum_of_numerator = [[0.],[0.]]
        sum_of_denominator = [[0.],[0.]]
        for i in range(2):
            for j in range(100):
                sum_of_numerator[i] += C[k][j]*a[j][i]*y[k][j]
                sum_of_denominator[i] += C[k][j]*y[k][j]
                if(sum_of_denominator[i] != 0):
                    M[k][i] = sum_of_numerator[i] / sum_of_denominator[i]
    
    #calculating new objective value
    for i in range(50):
        for j in range(100):
            d = squared_oclid_distance(M[i], a[j])
            Z_ALA_new += y[i][j]*C[i][j]*d
    
    #checking whether our objective value is improved or not
    if((Z_ALA_new + eps) > Z_ALA):
        break
    else:
        Z_ALA = Z_ALA_new
        Z_ALA_new = 0.
    
    #assign customers to nearest(least costly) facilities
    for j in range(100):
        mincost = 100000000000
        for i in range(50):
            cost_i_j = squared_oclid_distance(M[i], a[j])*C[i][j]
            if(cost_i_j < mincost):
                mincost = cost_i_j
                for k in range(50):
                    if(y[k][j] == 1):
                        y[k][j] = 0
                    if(k == i):
                        y[k][j] = 1
                    
                    


In [109]:
print(Z_ALA_new)
print(Z_ALA)

2454683.137567267
2454683.137567267


This was for one trial. Our objective value improved a lot by this algorithm. 

So we can run the algorithm for 1000 trial, and see the outputs.

In [135]:
all_Z_ALA = []
best_Z_ALA = 100000000000
best_y_ALA = np.zeros(shape = (50,100))
for d in range(1000):
    rn_list = []
    initialization()

    y = np.zeros(shape = (50,100))

    for i in range(len(rn_list)):
        k = rn_list[i]
        y[k][i] = 1.

    M = np.zeros(shape = (50,2)) #coordinates of the facilities

    # solving m single facility location problem 

    for k in range(50): 
        sum_of_numerator = [[0.],[0.]]
        sum_of_denominator = [[0.],[0.]]
        for i in range(2):
            for j in range(100):
                sum_of_numerator[i] += C[k][j]*a[j][i]*y[k][j]
                sum_of_denominator[i] += C[k][j]*y[k][j]
                if(sum_of_denominator[i] != 0):
                    M[k][i] = sum_of_numerator[i] / sum_of_denominator[i]

    # putting facilities which does not have any customers in (50,50)
    for i in range(len(M)):
        if(M[i][0] == 0 and M[i][1] == 0):
            M[i][0] = 50.
            M[i][1] = 50.

    # Calculating first objective value
    Z_ALA = 0.
    for i in range(len(C)):
        for j in range(len(C[0])):
            if(M[i][0] != 0):
                d = squared_oclid_distance(M[i], a[j])
            else:
                d = 0
            Z_ALA += y[i][j]*C[i][j]*d


    #assign customers to nearest(least costly) facilities
    for j in range(len(C[0])):
        mincost = 100000000000
        for i in range(len(C)):
            cost_i_j = squared_oclid_distance(M[i], a[j])*C[i][j]
            if(cost_i_j < mincost):
                mincost = cost_i_j
                for k in range(len(C)):
                    if(y[k][j] == 1):
                        y[k][j] = 0
                    if(k == i):
                        y[k][j] = 1

    eps = 0.001
    Z_ALA_new = 0.
    while(1):


        # solving m single facility location problem 
        for k in range(50): 
            sum_of_numerator = [[0.],[0.]]
            sum_of_denominator = [[0.],[0.]]
            for i in range(2):
                for j in range(100):
                    sum_of_numerator[i] += C[k][j]*a[j][i]*y[k][j]
                    sum_of_denominator[i] += C[k][j]*y[k][j]
                    if(sum_of_denominator[i] != 0):
                        M[k][i] = sum_of_numerator[i] / sum_of_denominator[i]

        #calculating new objective value
        for i in range(50):
            for j in range(100):
                d = squared_oclid_distance(M[i], a[j])
                Z_ALA_new += y[i][j]*C[i][j]*d

        #checking whether our objective value is improved or not
        if((Z_ALA_new + eps) > Z_ALA):
            break
        else:
            Z_ALA = Z_ALA_new
            Z_ALA_new = 0.

        #assign customers to nearest(least costly) facilities
        for j in range(100):
            mincost = 100000000000
            for i in range(50):
                cost_i_j = squared_oclid_distance(M[i], a[j])*C[i][j]
                if(cost_i_j < mincost):
                    mincost = cost_i_j
                    for k in range(50):
                        if(y[k][j] == 1):
                            y[k][j] = 0
                        if(k == i):
                            y[k][j] = 1

    if (Z_ALA < best_Z_ALA):
        best_Z_ALA = Z_ALA
        best_y_ALA = y
        best_M_ALA = M
    all_Z_ALA.append(Z_ALA)

In [136]:
Z_sum = 0
for i in range(1000):
    Z_sum += all_Z_ALA[i]
Z_average = Z_sum/1000

Our average objective value after 1000 trial is:

In [137]:
print(Z_average)

2756053.930709475


Our best objective value after 1000 trial is:

In [138]:
print(best_Z_ALA)

1312005.2773452483


Best allocation to the facilities is: (Here 1's represents customer j is allocated to facility i)

In [139]:
best_y_ALA = pd.DataFrame(best_y_ALA)
best_y_ALA

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Best facility location after 1000 trial is:

In [140]:
best_M_ALA = pd.DataFrame(best_M_ALA)
best_M_ALA

,0,1
0,66.202662,50.094840
1,87.887741,95.356450
2,78.993872,5.105181
3,58.860124,19.119931
4,66.665584,74.963271
5,64.812063,11.645246
6,49.199325,78.045478
7,48.670486,6.817051
8,78.735009,52.713221
9,96.166199,13.174740


### Question 4

In order to solve m facility location problem, we need to reform weiszfeld function and called it `ALA_weiszfeld_dynamic`.

Also in order to prevent dividing by 0, we assign `eps = 0.1`.

In [141]:
eps = 0.1
def ALA_weiszfeld_dynamic(T, default_val_x, default_val_y): 
    ALA_weiszfeld_x = np.zeros(shape = (50,T, 2))
    for i in range(50):
        ALA_weiszfeld_x[i][0][0] = default_val_x
        ALA_weiszfeld_x[i][0][1] = default_val_y
    for i in range(50):
        for t in range(1, T):
            for k in range(len(ALA_weiszfeld_x[0][0])):
                nom = 0
                denom = 0
                for j in range(100):
                    d = oclid_distance(ALA_weiszfeld_x[i][t-1], a[j])
                    if(d != 0):
                        nom += (C[i][j]) * (a[j][k]) * y[i][j] / d
                        denom += (C[i][j] * y[i][j]) / d
                if(denom == 0):
                    denom = denom + eps
                ALA_weiszfeld_x[i][t][k] = nom / denom                                                   
    return ALA_weiszfeld_x

Below, we only changed the single facility allocation algorithm to weiszfeld algorithm and run it for 1000 times.

- we let the algorithm to do 10 iteration (T = 10)
- and initial position of the facilities are (50,50)


In [183]:
all_Z = []
best_Z = 100000000000
best_y = np.zeros(shape = (50,100))
T = 10
for d in range(1000):
    rn_list = []
    initialization()

    y = np.zeros((len(C),len(C[0])))

    for i in range(len(rn_list)):
        k = rn_list[i]
        y[k][i] = 1.

    # solving m single facility location problem with Weiszfeld Algorithm
    M = ALA_weiszfeld_dynamic(10, 50., 50.)

    # Calculating first objective value
    Z = 0.
    for i in range(50):
        for j in range(100):
            if(M[i][T-1][0] != 0):
                d = oclid_distance(M[i][T-1], a[j])
            else:
                d = 0
            Z += y[i][j]*C[i][j]*d


    #assign customers to nearest(least costly) facilities
    for j in range(len(C[0])):
        mincost = 100000000000
        for i in range(len(C)):
            cost_i_j = oclid_distance(M[i][T-1], a[j])*C[i][j]
            if(cost_i_j < mincost):
                mincost = cost_i_j
                for k in range(len(C)):
                    if(y[k][j] == 1):
                        y[k][j] = 0
                    if(k == i):
                        y[k][j] = 1

    eps = 0.001
    Z_new =0.
    while(1):


        # solving m single facility location problem 
        M = ALA_weiszfeld_dynamic(10, 50., 50.)
        
        #calculating new objective value
        for i in range(50):
            for j in range(100):
                d = oclid_distance(M[i][T-1], a[j])
                Z_new += y[i][j]*C[i][j]*d

        #checking whether our objective value is improved or not
        if((Z_new + eps) > Z):
            break
        else:
            Z = Z_new
            Z_new = 0.

        #assign customers to nearest(least costly) facilities
        for j in range(100):
            mincost = 100000000000
            for i in range(50):
                cost_i_j = oclid_distance(M[i][T-1], a[j])*C[i][j]
                if(cost_i_j < mincost):
                    mincost = cost_i_j
                    for k in range(50):
                        if(y[k][j] == 1):
                            y[k][j] = 0
                        if(k == i):
                            y[k][j] = 1

    if (Z < best_Z):
        best_Z = Z
        best_y = y
        best_M = M
    all_Z.append(Z)

In [184]:
Z_sum_weiszfeld = 0
for i in range(1000):
    Z_sum_weiszfeld += all_Z[i]
Z_average_weiszfeld = Z_sum_weiszfeld/1000

Our average objective value after 1000 trial is:

In [228]:
print(Z_average_weiszfeld)

408764.25139203604


Our best objective value after 1000 trial is:

In [227]:
print(best_Z)

264137.1150606022


Best locations of the facilities are:

In [220]:
best_M_revised = np.zeros(shape = (50,2))

for i in range(50):
    for k in range(2):
        best_M_revised[i][k] = best_M[i][9][k]

In [221]:
for k in range(2):
    for i in range(50):
        if(best_M_revised[i][k] == 0):
            for j in range(100):
                if(best_y[i][j] == 1):
                    best_M_revised[i][k] = a[j][k]

In [222]:
best_M_revised = pd.DataFrame(best_M_revised)
best_M_revised

,0,1
0,27.272517,3.590124
1,41.424743,48.750522
2,0.000000,0.000000
3,0.000000,0.000000
4,91.633158,46.512227
5,26.612634,35.442645
6,66.202589,50.094727
7,77.858783,86.801243
8,20.932165,48.473123
9,0.000000,0.000000


Best allocation after 1000 trial is:

In [223]:
best_y_df = pd.DataFrame(best_y)
best_y_df

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In order to see better the allocations and facility locations, we import the data as .csv file:

In [226]:
best_y_df.to_csv(r'C:\Users\Yusuf Sina Öztürk\Desktop\7. Semester\IE 440\HW2\allocations_Q4.csv', index = False, header=False)
best_M_revised.to_csv(r'C:\Users\Yusuf Sina Öztürk\Desktop\7. Semester\IE 440\HW2\facility_locations_Q4.csv', index = False, header=False)
best_y_ALA.to_csv(r'C:\Users\Yusuf Sina Öztürk\Desktop\7. Semester\IE 440\HW2\allocations_Q3.csv', index = False, header=False)
best_M_ALA.to_csv(r'C:\Users\Yusuf Sina Öztürk\Desktop\7. Semester\IE 440\HW2\facility_locations_Q3.csv', index = False, header=False)

### Conclusion

When we compare single facility location problems (Question 1 and 2), objective value is way better in Weiszfeld algorithm. 

Also when we compare m facility location problems (Question 3 and 4), objective value is also way better when we used Weiszfeld algorithm in ALA algorithm.

Here, we connclude that Weiszfeld algortihm is very effective for both single facility allocation problem and m facility allocation problems. 